# Imports

In [212]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna
from copy import deepcopy
from pyDeepInsight import ImageTransformer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor

from scipy.spatial.distance import cdist

In [213]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [214]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)

## PCA Feature Selection

In [215]:
def pca_feature_selection(X, k, explained_variance_threshold=0.95):

    pca = PCA()
    pca.fit(X)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(cumulative_variance >= explained_variance_threshold) + 1

    pca = PCA(n_components=n_components)
    pca.fit(X)

    feature_importance = np.abs(pca.components_).sum(axis=0)
    top_k_indices = np.argsort(feature_importance)[-k:]
    
    return top_k_indices

k = 31
top_features_indices = pca_feature_selection(X_train, k)

In [216]:
y_train = np.where(y_train == 6, 0, 1)
y_test = np.where(y_test == 6, 0, 1)
y_val = np.where(y_val == 6, 0, 1)

X_train_selected = X_train[:, top_features_indices]
X_test_selected = X_test[:, top_features_indices]
X_val_selected = X_val[:, top_features_indices]

## PyDeepInsight

In [217]:
it = ImageTransformer(
    pixels=8,
    feature_extractor='tsne',
    discretization='lsa'
)

In [218]:
it.fit(X_train_selected)
X_train_images = it.transform(X_train_selected, 'pytorch')

X_test_images = it.transform(X_test_selected, 'pytorch')

X_val_images = it.transform(X_val_selected, 'pytorch')

## MAE

In [219]:
num_samples, channels, img_height, img_width = X_train_images.shape
latent_dim = 16

In [220]:
class Encoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(img_channels, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32 * 2 * 2, feature_dim)

    def forward(self, x):
        x = self.relu(self.conv1(x))  # Output: (batch_size, 16, 8, 8)
        x = self.pool(x)              # Output: (batch_size, 16, 4, 4)
        x = self.relu(self.conv2(x))  # Output: (batch_size, 32, 4, 4)
        x = self.pool(x)              # Output: (batch_size, 32, 2, 2)
        x = x.view(x.size(0), -1)     # Flatten to (batch_size, 128)
        x = self.fc1(x)               # Output: (batch_size, feature_dim)
        return x

In [221]:
class Decoder(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32):
        super(Decoder, self).__init__()
        self.fc2 = nn.Linear(feature_dim, 32 * 2 * 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.deconv1 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(16, img_channels, kernel_size=3, stride=1, padding=1)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, z):
        x = self.relu(self.fc2(z))           # Output: (batch_size, 128)
        x = x.view(x.size(0), 32, 2, 2)      # Reshape to (batch_size, 32, 2, 2)
        x = self.upsample(x)                 # Upsample to (batch_size, 32, 4, 4)
        x = self.relu(self.deconv1(x))       # Output: (batch_size, 16, 4, 4)
        x = self.upsample(x)                 # Upsample to (batch_size, 16, 8, 8)
        x = self.sigmoid(self.deconv2(x))    # Output: (batch_size, img_channels, 8, 8)
        return x

In [222]:
class MAE(nn.Module):
    def __init__(self, img_channels=1, feature_dim=32, latent_dim=2):
        super(MAE, self).__init__()
        self.encoder = Encoder(img_channels, feature_dim, latent_dim)
        self.decoder = Decoder(img_channels, feature_dim)

    def mask_input(self, x, mask_ratio=0.25):
        # Generate a mask with 0s and 1s, keeping only (1-mask_ratio) of the original input
        mask = torch.rand(x.shape, device=x.device) > mask_ratio
        x_masked = x * mask
        return x_masked, mask

    def forward(self, x, mask_ratio=0.25):
        x_masked, mask = self.mask_input(x, mask_ratio)  # Apply masking to input
        z = self.encoder(x_masked)
        reconstructed = self.decoder(z)
        return reconstructed, mask

In [223]:
def mae_loss_function(reconstructed, original, mask):
    # Only calculate reconstruction loss on the masked parts
    masked_original = original * mask
    reconstruction_loss = F.mse_loss(reconstructed, masked_original, reduction='sum')
    return reconstruction_loss

In [224]:
model = MAE(img_channels=3, feature_dim=32, latent_dim=16).to(device)
model.load_state_dict(torch.load("deepinsight_mae_normal.pth"))

<All keys matched successfully>

In [225]:
normal_indices = np.where(y_train == 0)[0]
X_train_normal = X_train_images[normal_indices]
y_train_normal = y_train[normal_indices]

anomaly_indices = np.where(y_train == 1)[0]
X_train_anomaly = X_train_images[anomaly_indices]
y_train_anomaly = y_train[anomaly_indices]

X_train_normal_tensor = torch.tensor(X_train_normal, dtype=torch.float32)
X_train_anomaly_tensor = torch.tensor(X_train_anomaly, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_images, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_images, dtype=torch.float32)

train_normal_dataset = TensorDataset(X_train_normal_tensor)
train_anomaly_dataset = TensorDataset(X_train_anomaly_tensor)
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.long))
val_dataset = TensorDataset(X_val_tensor, torch.tensor(y_val, dtype=torch.long))

batch_size = 32 
train_normal_loader = DataLoader(train_normal_dataset, batch_size=batch_size, shuffle=False)
train_anomaly_loader = DataLoader(train_anomaly_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_340/740995774.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_normal_tensor = torch.tensor(X_train_normal, dtype=torch.float32)
/tmp/ipykernel_340/740995774.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_anomaly_tensor = torch.tensor(X_train_anomaly, dtype=torch.float32)
/tmp/ipykernel_340/740995774.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test_images, dtype=torch.float32)
/tmp/ipykernel_340/740995774.py:12: UserWarning: To copy construct from a tensor, it

In [226]:
def extract_latent_features(model, data_loader, device='cuda'):
    model.eval() 
    latent_features = []  

    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader), desc="Extracting features"):
            if len(batch) == 2:
                data, _ = batch  
            else:
                (data,) = batch  
            
            data = data.to(device)

            latent_feature = model.encoder(data)
            latent_features.append(latent_feature.cpu().numpy())

    latent_features = np.concatenate(latent_features, axis=0)
    
    return latent_features

In [227]:
train_normal_latent_features = extract_latent_features(model, train_normal_loader, device)
train_anomaly_latent_features = extract_latent_features(model, train_anomaly_loader, device)
val_latent_features = extract_latent_features(model, val_loader, device)
test_latent_features = extract_latent_features(model, test_loader, device)

Extracting features: 100%|██████████| 1611/1611 [00:01<00:00, 1167.24it/s]


In [228]:
train_normal_latent_features.shape, y_train_normal.shape

((59520, 32), (59520,))

In [229]:
train_anomaly_latent_features.shape, y_train_anomaly.shape

((105390, 32), (105390,))

In [230]:
val_latent_features.shape, y_val.shape

((41228, 32), (41228,))

In [231]:
test_latent_features.shape, y_test.shape

((51535, 32), (51535,))

# Important Notes

- SAFE has 4 modules. We are assuming that the MAE is pre-trained already and we have already used the encoder head to extract latent features. We are implementing this on the LOF in module 4 (novelty detector)

## Creation functions

In [232]:
def create_phi(normal_data, c):
    """
    Concept creation function for normal data.
    Uses k-Means clustering to partition normal data into c clusters.
    
    Args:
        normal_data (numpy array): The normal data points.
        c (int): Number of desired normal concepts.
    
    Returns:
        list of numpy arrays: List of normal clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(normal_data)
    
    normal_concepts = [normal_data[labels == i] for i in range(c)]
    print("Finished creating normal concepts")
    
    return normal_concepts


def create_gamma(anomaly_data, c):
    """
    Concept creation function for anomaly data.
    Uses k-Means clustering to partition anomaly data into c clusters.
    
    Args:
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired anomaly concepts.
    
    Returns:
        list of numpy arrays: List of anomaly clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(anomaly_data)
    
    anomaly_concepts = [anomaly_data[labels == i] for i in range(c)]
    print("Finished creating anomaly concepts")
    
    return anomaly_concepts
    
def match_lambda(anomaly_concepts, normal_concepts):
    """
    Matches each normal concept with the closest anomaly concept.
    Uses Euclidean distance to determine the best match.
    
    Args:
        anomaly_concepts (list of numpy arrays): List of anomaly clusters.
        normal_concepts (list of numpy arrays): List of normal clusters.
    
    Returns:
        list of tuples: Pairs of (normal_concept, matched_anomaly_concept)
    """
    pairs = []
    remaining_anomalies = anomaly_concepts.copy()

    for normal_concept in normal_concepts:
        normal_centroid = np.mean(normal_concept, axis=0)
        anomaly_centroids = [np.mean(ac, axis=0) for ac in remaining_anomalies]

        distances = cdist([normal_centroid], anomaly_centroids, metric='euclidean')[0]
        closest_idx = np.argmin(distances)

        pairs.append((normal_concept, remaining_anomalies[closest_idx]))
        remaining_anomalies.pop(closest_idx)

    print("Finished matching concept pairs")
    
    return pairs

## Evaluation metrics

In [233]:
def lifelong_roc_auc(R):
    """
    Computes the Lifelong ROC-AUC metric.
    
    Args:
        R (numpy array): NxN matrix of ROC-AUC scores, where R[i, j] is the model's 
                         performance on concept j after learning concept i.
    
    Returns:
        float: Lifelong ROC-AUC score.
    """
    N = R.shape[0]
    lower_triangular_sum = np.sum(np.tril(R))
    normalization_factor = (N * (N + 1)) / 2

    return lower_triangular_sum / normalization_factor

def BWT(R):
    """
    Computes the Backward Transfer (BWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: BWT score.
    """
    N = R.shape[0]
    backward_transfer = 0
    count = 0

    for i in range(1, N):
        for j in range(i):
            backward_transfer += (R[i, j] - R[j, j])
            count += 1

    return backward_transfer / count if count > 0 else 0

def FWT(R):
    """
    Computes the Forward Transfer (FWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: FWT score.
    """
    N = R.shape[0]
    forward_transfer = 0
    count = 0

    for i in range(N):
        for j in range(i + 1, N): 
            forward_transfer += R[i, j]
            count += 1

    return forward_transfer / count if count > 0 else 0 

## Scenario Design + Evaluation Protocol

In [234]:
def scenario_design(normal_data, anomaly_data, c):
    """
    Implements Algorithm 1 to create a lifelong learning scenario.
    
    Args:
        normal_data (numpy array): The normal data points.
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired concepts.
    
    Returns:
        list of tuples: List of (normal_concept, anomaly_concept) pairs forming the scenario.
    """
    normal_concepts = create_phi(normal_data, c)
    anomaly_concepts = create_gamma(anomaly_data, c)
    
    scenario = match_lambda(anomaly_concepts, normal_concepts)
    
    return scenario

def evaluation_protocol(T, E, Y, strategy="naive", replay_buffer_size=3000):
    """
    Implements Algorithm 2: Lifelong Learning Evaluation Protocol with multiple strategies.
    
    Args:
        T (list): Sequence of N training sets.
        E (list): Sequence of N testing sets.
        Y (list): Sequence of true labels for test sets.
        strategy (str): Strategy for training ("naive", "mste", "replay").
        replay_buffer_size (int): Maximum size of replay buffer (only used if strategy="replay").

    Returns:
        numpy array: NxN results matrix R where R[i, j] is ROC-AUC of model on E[j] after learning T[i].
    """
    N = len(T)
    R = np.zeros((N, N))  

    if strategy == "mste":
        experts = []
    
    elif strategy == "replay":
        replay_buffer = []

    
    for i, Ti in tqdm(enumerate(T), desc=f"Evaluating using {strategy} strategy"):
        if strategy == "naive":
            model = LocalOutlierFactor(n_neighbors=20, novelty=True)  
            model.fit(Ti)

        """
        elif strategy == "mste":
            new_model = LocalOutlierFactor(n_neighbors=20, novelty=True)  
            new_model.fit(Ti)
            experts.append(new_model)
        """
        
        elif strategy == "replay":
            if replay_buffer:
                combined_data = np.vstack((np.array(replay_buffer), Ti))
            else:
                combined_data = Ti

            model = LocalOutlierFactor(n_neighbors=20, novelty=True)  
            model.fit(combined_data)

            replay_buffer.extend(Ti)
            
            if len(replay_buffer) > replay_buffer_size:
                replay_buffer = replay_buffer[-replay_buffer_size:]

        for j, ((Ej_normal, Ej_anomaly), (y_normal, y_anomaly)) in enumerate(zip(E, Y)):
           
            test_data = np.vstack((Ej_normal, Ej_anomaly))
            test_labels = np.hstack((y_normal, y_anomaly))  
        
            scores = -model.decision_function(test_data)  
            R[i, j] = roc_auc_score(test_labels, scores)
            
    return R

# Experiments

### First concatenate all our X_ and y data together

In [284]:
X = np.vstack((train_normal_latent_features, train_anomaly_latent_features, val_latent_features, test_latent_features))
y = np.hstack((y_train_normal, y_train_anomaly, y_val, y_test))

### Create 'c' normal/anomaly concepts

In [285]:
num_concepts = 5

X_normal = X[y == 0]  
X_anomaly = X[y == 1]

normal_concepts = create_phi(X_normal, num_concepts)
anomaly_concepts = create_gamma(X_anomaly, num_concepts)

Finished creating normal concepts
Finished creating anomaly concepts


### Use lambda function to pair normal/anomaly concepts 

In [286]:
concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

Finished matching concept pairs


### Creating training and testing sets

In [287]:
T = []  
E = [] 
Y = []

for normal, anomaly in concept_pairs:

    normal_train, normal_test = train_test_split(normal, test_size=0.3, random_state=42)
    anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.3, random_state=42)  

    T.append(normal_train)
    E.append((normal_test, anomaly_test))

    y_normal_test = np.zeros(len(normal_test))
    y_anomaly_test = np.ones(len(anomaly_test))
    
    Y.append((y_normal_test, y_anomaly_test))

In [288]:
"""
normal_concepts = create_phi(train_normal_latent_features, num_concepts)
#anomaly_concepts = create_gamma(train_anomaly_latent_features, num_concepts)
#concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

T = normal_concepts

E_normal = create_phi(test_latent_features, num_concepts)
E_anomaly = create_gamma(test_latent_features, num_concepts)   

Y_normal = [y_test[:len(normal)] for normal in E_normal]  
Y_anomaly = [y_test[:len(anomaly)] for anomaly in E_anomaly]  

E = [(normal, anomaly) for normal, anomaly in zip(E_normal, E_anomaly)]
Y = [(y_normal, y_anomaly) for y_normal, y_anomaly in zip(Y_normal, Y_anomaly)]
"""

'\nnormal_concepts = create_phi(train_normal_latent_features, num_concepts)\n#anomaly_concepts = create_gamma(train_anomaly_latent_features, num_concepts)\n#concept_pairs = match_lambda(anomaly_concepts, normal_concepts)\n\nT = normal_concepts\n\nE_normal = create_phi(test_latent_features, num_concepts)\nE_anomaly = create_gamma(test_latent_features, num_concepts)   \n\nY_normal = [y_test[:len(normal)] for normal in E_normal]  \nY_anomaly = [y_test[:len(anomaly)] for anomaly in E_anomaly]  \n\nE = [(normal, anomaly) for normal, anomaly in zip(E_normal, E_anomaly)]\nY = [(y_normal, y_anomaly) for y_normal, y_anomaly in zip(Y_normal, Y_anomaly)]\n'

In [289]:
R_naive = evaluation_protocol(T, E, Y, strategy="naive")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:06,  1.21s/it]

Lifelong ROC-AUC: 0.5209079981103106, BWT: -0.598555385482603, FWT: 0.5675865361608492


In [290]:
"""
R_mste = evaluation_protocol(T, E, Y, strategy="mste")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_mste)}, BWT: {BWT(R_mste)}, FWT: {FWT(R_mste)}")
"""

'\nR_mste = evaluation_protocol(T, E, Y, strategy="mste")\nprint(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_mste)}, BWT: {BWT(R_mste)}, FWT: {FWT(R_mste)}")\n'

In [291]:
R_replay = evaluation_protocol(T, E, Y, strategy="replay", replay_buffer_size=3000)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:07,  1.45s/it]

Lifelong ROC-AUC: 0.7456418835718264, BWT: -0.24209173663826164, FWT: 0.4489903474284862
